In [2]:
import plotly.graph_objects as go
import plotly.io as pio
import os
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../../data/job_data.csv')

In [5]:
df.head(10)

,Unnamed: 0,title,company_name,location,via,description,schedule_type,salary,query,qualifications,responsibilities,benefits,degree,experience,remote,city,state
0,0,Ethereum Blockchain Developer (Remote),Ex Populus,Anywhere,Built In,Company Overview:\nEx Populus is a cutting-edg...,Full-time,NaN,Blockchain,['2-3 years of Software Development experience...,"['Design, maintain and deploy smart contracts ...",NaN,NaN,2.5,True,NaN,NaN
1,1,Blockchain Engineer,21.co,"New York, NY",Greenhouse,We are seeking a highly motivated and skilled ...,Full-time,180000.0,Blockchain,"[""Bachelor's or Master's degree in Computer Sc...","['As a Blockchain Engineer, you will be respon...",['(NYC only) Pursuant to Section 8-102 of titl...,Master's,NaN,False,New York,New York
2,2,Blockchain Course Instructor,Blockchain Institute of Technology,Anywhere,LinkedIn,"Are you a blockchain, cryptocurrency, NFT, Met...",Contractor,NaN,Blockchain,"['3+ years of experience in blockchain, crypto...",['Our expert technical team will provide the s...,NaN,NaN,3.0,True,NaN,NaN
3,3,Python based - Blockchain developer to join ex...,Upwork,Anywhere,Upwork,Need someone to join our existing team to spee...,Contractor,41600.0,Blockchain,"['Candidates must be willing to sign, non-disc...",['Will discuss details with the selected candi...,NaN,NaN,NaN,True,NaN,NaN
4,4,Blockchain DevOps Engineer (Remote),Telnyx,United States,Startup Jobs,"About Telnyx\n\nAt Telnyx, we’re architecting ...",Full-time,NaN,Blockchain,['You are a highly motivated and experienced B...,['To build a best-in-class Filecoin (FIL) Mini...,NaN,Bachelor's,NaN,True,NaN,NaN
5,5,Ethereum Developer Remote US (Blockchain-DeFi),Turnblock.io,Anywhere,ZipRecruiter,Our client is on crypto’s cutting edge technol...,Full-time,150000.0,Blockchain,"['5+ years of development experience', 'Experi...","['As a member of an agile engineering team, yo...",['Great compensation package'],NaN,5.0,True,NaN,NaN
6,6,Social Media Marketing (Blockchain/Crypto),Bitquery,Anywhere,AngelList,• Maintain a social media calendar\n• Build Bi...,Full-time,NaN,Blockchain,['English Proficiency (Written and Speaking) w...,['Put our Bitquery updates and educational con...,['Opportunity to work & collaborate with a tru...,NaN,NaN,True,NaN,NaN
7,7,Senior Software Engineer - Blockchain Network ...,Jobot,Anywhere,Dice,"competitive salaries, stock options, company p...",Full-time,175000.0,Blockchain,['At least 5 years of experience with designin...,"['Rotating breakfast menu served daily', 'Dinn...","['Salary: $150,000 - $200,000 per year', 'Our ...",NaN,5.0,True,NaN,NaN
8,8,Blockchain Developer,Atechstar,United States,OPTnation,Requirements: Strong software development back...,Full-time,141000.0,Blockchain,['Requirements: Strong software development ba...,NaN,NaN,NaN,NaN,True,NaN,NaN
9,9,DeFi Blockchain Co-Founder,Cryptops Exchange,Anywhere,LinkedIn,Cryptops is currently looking for a Co-Founder...,Full-time,NaN,Blockchain,['Can personally invest or immediately support...,['Other decentralized derivatives exchanges re...,NaN,NaN,NaN,True,NaN,NaN


In [6]:
df.dtypes

Unnamed: 0            int64
title                object
company_name         object
location             object
via                  object
description          object
schedule_type        object
salary              float64
query                object
qualifications       object
responsibilities     object
benefits             object
degree               object
experience          float64
remote                 bool
city                 object
state                object
dtype: object

In [7]:
# create a copy of the dataframe
df_copy = df.copy()

# filter out rows with NA or 0 values
df_copy = df_copy.dropna(subset=['salary', 'experience'])
df_copy = df_copy[(df_copy['salary'] > 0) & (df_copy['experience'] > 0)]

# round the 'salary' column to the nearest 1000
df_copy['salary'] = np.round(df_copy['salary'] / 1000) * 1000

# round the 'experience' column to the nearest year
df_copy['experience'] = np.round(df_copy['experience']).astype('int64')

# group by state and calculate the sum of salary
salary_by_state = df_copy.groupby('state')['salary'].mean().reset_index()

# group by state and calculate the mean of experience
experience_by_state = df_copy.groupby('state')['experience'].mean().reset_index()


In [8]:
experience_by_state.dtypes

state          object
experience    float64
dtype: object

In [9]:
experience_by_state['experience'] = experience_by_state['experience'].round().astype(int) # round to whole number

In [10]:
salary_by_state['salary'] = salary_by_state['salary'].round(-3) # round to nearest 1000

In [13]:
salary_format = lambda x: '${:,.0f}'.format(x)
salary_by_state['salary'] = pd.to_numeric(salary_by_state['salary']).apply(salary_format)

ValueError: Unable to parse string "$152,000" at position 0

In [42]:
# create the traces
trace1 = go.Table(
    header=dict(values=['<b>State', '<b>Average Experience Required (Years)'], fill_color='crimson'),
    cells=dict(values=[experience_by_state['state'], experience_by_state['experience']]),
    visible=True,
    
)

trace2 = go.Table(
    header=dict(values=['<b>State', '<b>Average Annual Salary (USD)'], fill_color='crimson', font=dict(family='Arial', size=13)),
    cells=dict(values=[salary_by_state['state'], salary_by_state['salary']]),
    visible=False,
)


# Create the text annotation
text_annotation = go.layout.Annotation(
    text='<b>Hello',
    font=dict(family='Arial', size=12),
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1,
    y=1,
    xanchor='left',
    yanchor='bottom'
)

# create the updatemenus list and buttons for switching between tables
updatemenus = [
    dict(
        type='buttons',
        buttons=list([
            dict(label='<b>Average Experience',
                 method='update',
                 args=[{'visible': [True, False]},
                       {'title': '<b>Average Experience by State'}]),
            dict(label='<b>Average Salary',
                 method='update',
                 args=[{'visible': [False, True]},
                       {'title': '<b>Average Salary by State'}])
        ]),
        x=1,
        y=1.2,
        direction='left',
        pad=dict(r=10, t=10),
        showactive=True,
        bgcolor='#d3d3d3',
    )
]

# set the layout for the plot
layout = dict(title='<b>Average Experience by State',
              updatemenus=updatemenus)

# create the figure and add the traces
fig = go.Figure(data=[trace1, trace2], layout=layout)

# display the plot
fig.show()


pio.write_html(fig, file="../../website/plots/table.html")
